<a href="https://colab.research.google.com/github/chakshu-agarwal/Investments-Summary/blob/main/Robinhood_Data_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install pandasql
# pip install robin_stocks

In [132]:
import robin_stocks as r
from robin_stocks.globals import LOGGED_IN, SESSION, OUTPUT
import pyotp
import requests
import json
import pandas as pd
import pandasql as ps
from datetime import datetime

In [ ]:
totp  = pyotp.TOTP("[MFA Key]").now()
print("Current OTP:", totp)
login = r.login('[Email]','[Password]', mfa_code=totp)

In [ ]:
"""Takes a stock instrument url and returns the ticker symbol associated with the stock.
    :param id: A string that represents the stocks instrument url.
    :type id: str
    :returns:  The symbol.
    """
symbol_for_stock = lambda url: requests.get(url).json()['symbol']

In [ ]:
# Raw dataset with all filled orders

filled_orders = []
filled_orders.append(['symbol','latest_price', 'order_type','quantity','unit_price','total_price','fees','timestamp'])
for order in raw:
  if order['state'] == 'filled':
    order_details = []
    order_details.append(symbol_for_stock(order['instrument']))
    try:
      price = r.stocks.get_latest_price(symbol_for_stock(order['instrument']), includeExtendedHours=True)[0]
    except:
      price = '0'
    order_details.append(price)
    order_details.append(order['side'])
    order_details.append(order['cumulative_quantity'])
    order_details.append(order['average_price'])
    order_details.append(order['executed_notional']['amount'])
    order_details.append(order['fees'])
    order_details.append(order['updated_at'])
    filled_orders.append(order_details)

# Converting to dataframe for analysis
order_data = pd.DataFrame(filled_orders[1:], columns=filled_orders[0])
order_data

In [ ]:
# Summary Information on all filled orders by Ticker Symbol

query = "WITH summary AS ( \
  SELECT symbol,\
  CAST(SUM(CASE WHEN order_type = 'buy' THEN quantity END) AS FLOAT) as buy_quantity,\
  CAST(SUM(CASE WHEN order_type = 'buy' THEN total_price END)/SUM(CASE WHEN order_type = 'buy' THEN quantity END) AS FLOAT) as avg_buy_price,\
  CAST(SUM(CASE WHEN order_type = 'buy' THEN total_price END) AS FLOAT) as money_out,\
  CAST(SUM(CASE WHEN order_type = 'sell' THEN quantity END) AS FLOAT) as sell_quantity,\
  CAST(SUM(CASE WHEN order_type = 'sell' THEN total_price END)/SUM(CASE WHEN order_type = 'sell' THEN quantity END) AS FLOAT) as avg_sell_price,\
  CAST(SUM(CASE WHEN order_type = 'sell' THEN total_price END) AS FLOAT) as money_in,\
  CAST(MAX(latest_price) AS FLOAT) as latest_price,\
  MAX(timestamp) as last_txn_time \
FROM order_data \
GROUP BY symbol \
) \
SELECT \
  *,\
  CAST(money_in - (sell_quantity*avg_buy_price) AS DECIMAL) AS realized_gains_loss,\
  CASE \
    WHEN buy_quantity != sell_quantity AND sell_quantity IS NOT NULL AND latest_price IS NOT NULL \
      THEN ((buy_quantity-sell_quantity)*latest_price) - ((buy_quantity-sell_quantity)*avg_buy_price) \
    WHEN sell_quantity IS NULL AND latest_price != 'None' \
      THEN (buy_quantity*latest_price) - (buy_quantity*avg_buy_price) \
    WHEN sell_quantity IS NULL AND latest_price IS NULL \
      THEN 0 - (buy_quantity*avg_buy_price) \
    ELSE 0 \
  END AS unrealized_gains_loss \
FROM summary;"
summary_all_orders =  ps.sqldf(query)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
summary_all_orders

In [ ]:
# Summary Key Results
print('Money Invested: ', summary_all_orders['money_out'].sum())
print('Money Return: ', summary_all_orders['money_in'].sum())
print('Realized Gain/Loss: ', summary_all_orders['realized_gains_loss'].sum())
print('UnRealized Gain/Loss: ', summary_all_orders['unrealized_gains_loss'].sum())


In [ ]:
# YTD calculation
current_year = "\"" + str(datetime.now().year) +"\""
query = "WITH dataset AS (SELECT A.symbol,\
  A.buy_quantity,\
  A.avg_buy_price,\
  A.money_out,\
  B.sell_quantity,\
  B.avg_sell_price,\
  B.money_in,\
  B.sell_txn_year \
FROM (SELECT symbol,\
    SUM(CASE WHEN order_type = 'buy' THEN quantity END) as buy_quantity,\
    SUM(CASE WHEN order_type = 'buy' THEN total_price END)/SUM(CASE WHEN order_type = 'buy' THEN quantity END) as avg_buy_price,\
    SUM(CASE WHEN order_type = 'buy' THEN total_price END) as money_out \
  FROM order_data WHERE order_type = 'buy' GROUP BY symbol) A \
LEFT JOIN (SELECT symbol,\
    SUM(CASE WHEN order_type = 'sell' THEN quantity END) as sell_quantity,\
    SUM(CASE WHEN order_type = 'sell' THEN total_price END)/SUM(CASE WHEN order_type = 'sell' THEN quantity END) as avg_sell_price,\
    SUM(CASE WHEN order_type = 'sell' THEN total_price END) as money_in,\
    strftime('%Y',timestamp) as sell_txn_year \
  FROM order_data WHERE order_type = 'sell' AND strftime('%Y',timestamp) == {} GROUP BY symbol) B \
ON A.symbol = B.symbol) \
SELECT symbol,\
  (sell_quantity*avg_buy_price) as money_out,\
  money_in,\
  (money_in)-(sell_quantity*avg_buy_price) as realized_gains_loss \
FROM dataset \
WHERE sell_quantity IS NOT NULL \
GROUP BY symbol;".format(current_year)

ytd_data =  ps.sqldf(query)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
ytd_data

In [181]:
# Summary Key Results
print('Realized Gain/Loss: ', ytd_data['realized_gains_loss'].sum())

Realized Gain/Loss:  -1167.4956650246295


In [ ]:
# Next Steps:

# 1. Account for stock splits and acquisitions in stock price
# 2. Account for repurchase of same stock at a later date
# 3. Tag investments by category (long terms - growth stocks, etf, short term - gambling money) to identify future investment split
# 4. Add other accounts (schwab)
# 5. Add UI layer